## Data Analysis

In [1]:
#!/usr/bin/env python3
"""
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def load_all_data():
    print("="*80)
    print("="*80)
    
    # 1. original data
    utterances = pd.read_csv('data/ncte_single_utterances.csv')  # ['speaker', 'text', 'year', 'OBSID', 'video_id', 'cleaned_text', 'num_words', 'turn_idx', 'comb_idx']
    paired = pd.read_csv('data/paired_annotations.csv') # ['exchange_idx', 'OBSID', 'student_text', 'teacher_text', 'student_on_task', 'teacher_on_task', 'high_uptake', 'focusing_question']
    reasoning = pd.read_csv('data/student_reasoning.csv') #['comb_idx', 'OBSID', 'NCTETID', 'text', 'student_reasoning', 'annotator_comment']
    
    # 2. CLASS observation
    classroom_obs = pd.read_csv('ICPSR_36095/DS0001/36095-0001-Data.tsv', sep='\t') 
    # ['NCTETID', 'DISTRICT', 'CLASS_11', 'CLASS_12', 'CLASS_13', 'SCHOOLYEAR_SP',
    #  'OBSID', 'RATERID', 'OBSYEAR', 'OBSMONTH', 'OBSDAY', 'CHAPNUM', 'CLPC', 'CLNC',
    #   'CLTS', 'CLRSP', 'CLBM', 'CLPRDT', 'CLILF', 'CLCU', 'CLAPS', 'CLQF', 'CLINSTD', 'CLSTENG']
    
    # 3. teacher value added
    teacher_va = pd.read_csv('ICPSR_36095/DS0004/36095-0004-Data.tsv', sep='\t')
    # ['DISTRICT', 'NCTETID', 'STATEVA_M', 
    
    # 4. MQI
    mqi_data = pd.read_csv('ICPSR_36095/DS0002/36095-0002-Data.tsv', sep='\t')
    # ['NCTETID', 'DISTRICT', 'MQI5','OBSID', 'RATERID',
    #    'SEGMENT', 'SCHOOLYEAR_SP', 'OBSYEAR', 'OBSMONTH', 'OBSDAY', 'CHAPNUM',
    
    # 5. student metadata (DS0005)
    student_meta = pd.read_csv('ICPSR_36095/DS0005/36095-0005-Data.tsv', sep='\t')
    #['DISTRICT', 'SCHOOLID', 'NCTETID', 'CLASS_ID_M', 'NCTESID',
    
    # 6. teacher metadata (DS0006)  
    teacher_meta = pd.read_csv('ICPSR_36095/DS0006/36095-0006-Data.tsv', sep='\t')
    # print(teacher_meta.columns)
    # ['NCTETID', 'SURVEYYEAR_SP', 'EXPERIENCE', 
    
    print(f"data loaded:")
    print(f"- Utterances: {len(utterances):,}")
    print(f"- Paired annotations: {len(paired):,}")
    print(f"- Student reasoning: {len(reasoning):,}")
    print(f"- Classroom obs: {len(classroom_obs):,}")
    print(f"- Teacher VA: {len(teacher_va):,}")
    print(f"- MQI data: {len(mqi_data):,}")
    print(f"- Student metadata: {len(student_meta):,}")
    print(f"- Teacher metadata: {len(teacher_meta):,}")


    print(f"unique observation ID (OBSID) in utterances: {utterances['OBSID'].nunique()}")
    print(f"unique observation ID (OBSID) in paired: {paired['OBSID'].nunique()}")
    print(f"unique observation ID (OBSID) in student_reasoning: {reasoning['OBSID'].nunique()}")
    print(f"unique teacher ID (NCTETID) in student_reasoning: {reasoning['NCTETID'].nunique()}")

    print(f"unique observation ID (OBSID) in classroom_obs: {classroom_obs['OBSID'].nunique()}")
    print(f"unique teacher ID (NCTETID) in classroom_obs: {classroom_obs['NCTETID'].nunique()}")
    
    print(f"unique observation ID (OBSID) in mqi: {mqi_data['OBSID'].nunique()}")
    print(f"unique teacher ID (NCTETID) in mqi: {mqi_data['NCTETID'].nunique()}")
    print(f"common observation ID (OBSID) in utterances and mqi: {len(set(utterances['OBSID']) & set(mqi_data['OBSID']))}")
    print(f"common observation ID (OBSID) in utterances and reasoning: {len(set(utterances['OBSID']) & set(reasoning['OBSID']))}")
    print(f"common observation ID (OBSID) in paired and reasoning: {len(set(paired['OBSID']) & set(reasoning['OBSID']))}")

    print(f"common combox_idx ID (comb_idx) in utterances and reasoning: {len(set(utterances['comb_idx']) & set(reasoning['comb_idx']))}")
    print(f"common combox_idx ID (comb_idx) in paired and reasoning: {len(set(paired['exchange_idx']) & set(reasoning['comb_idx']))}")


    print(f"common teacher ID (NCTETID) in student_reasoning and mqi: {len(set(reasoning['NCTETID']) & set(mqi_data['NCTETID']))}")
    print(f"common observation ID (OBSID) in student_reasoning and mqi: {len(set(reasoning['OBSID']) & set(mqi_data['OBSID']))}")

    # print(mqi_data.columns)
    return utterances, paired, reasoning, classroom_obs, teacher_va, mqi_data, student_meta, teacher_meta
utterances, paired, reasoning, classroom_obs, teacher_va, mqi_data, student_meta, teacher_meta = load_all_data()


# speaker: The speaker of the utterance.
# text: The utterance text.
# year: The school year in which transcript was taken. 1 = 2010-11, 2 = 2011-12, 3 = 2012-13 school year.
# OBSID: The unique ID for the transcript. Observation ID, mappable to unique transcripts in the NCTE dataset.
# video_id: The unique ID of the video from which the transcript was taken.
# cleaned_text: The cleaned version of text with removed punctuation and lower casing.
# num_words: Number of words in the utterance text.
# turn_idx: The utterance turn number in the transcript.
# comb_idx: The concatenation of OBSID and turn_idx, i.e., comb_idx = <OBSID>_<turn_idx>.

# CLPC（Positive Climate)
# CLRSP (Regard for Student Perspectives)
# CLTS (Teacher Sensitivity)
# CLINSTD (Instructional Dialogue)

# MQI5 (Teacher's Classroom Management Index)

# Value-added  map  year

# 1 = 2010-11, ->  STATEVA_M11
# 2 = 2011-12, ->  STATEVA_M12
# 3 = 2012-13, ->  STATEVA_M13



data loaded:
- Utterances: 580,408
- Paired annotations: 2,348
- Student reasoning: 2,000
- Classroom obs: 6,206
- Teacher VA: 314
- MQI data: 28,850
- Student metadata: 12,661
- Teacher metadata: 313
unique observation ID (OBSID) in utterances: 1660
unique observation ID (OBSID) in paired: 776
unique observation ID (OBSID) in student_reasoning: 744
unique teacher ID (NCTETID) in student_reasoning: 276
unique observation ID (OBSID) in classroom_obs: 1713
unique teacher ID (NCTETID) in classroom_obs: 317
unique observation ID (OBSID) in mqi: 1694
unique teacher ID (NCTETID) in mqi: 317
common observation ID (OBSID) in utterances and mqi: 1660
common observation ID (OBSID) in utterances and reasoning: 744
common observation ID (OBSID) in paired and reasoning: 364
common combox_idx ID (comb_idx) in utterances and reasoning: 2000
common combox_idx ID (comb_idx) in paired and reasoning: 61
common teacher ID (NCTETID) in student_reasoning and mqi: 276
common observation ID (OBSID) in student

## Preprocess

1. remove unvalid data from metadata (teacher/student...)
2. create observation dict (total_turns, student_turns, total_words, student_words)
3. teacher_val_dict (map to teacher year level)

In [2]:
# process data
def process_data(student_meta, teacher_meta, mqi_data, classroom_obs, teacher_va):
    student_meta.loc[student_meta['S_MALE'] == ' ', 'S_MALE'] = 0
    student_meta.loc[student_meta['S_AFAM'] == ' ', 'S_AFAM'] = 0
    student_meta.loc[student_meta['S_WHITE'] == ' ', 'S_WHITE'] = 0
    student_meta.loc[student_meta['S_HISP'] == ' ', 'S_HISP'] = 0
    student_meta.loc[student_meta['S_ASIAN'] == ' ', 'S_ASIAN'] = 0
    student_meta.loc[student_meta['S_RACE_OTHER'] == ' ', 'S_RACE_OTHER'] = 0
    student_meta.loc[student_meta['S_FRPL'] == ' ', 'S_FRPL'] = 0
    student_meta.loc[student_meta['S_SPED'] == ' ', 'S_SPED'] = 0
    student_meta.loc[student_meta['S_LEP'] == ' ', 'S_LEP'] = 0
    # remove 998, 999, value 998 and 999 are missing values
    student_meta = student_meta[~student_meta.S_MALE.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_AFAM.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_WHITE.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_HISP.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_ASIAN.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_RACE_OTHER.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_FRPL.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_SPED.isin(['998','999',999,998,' '])]
    student_meta = student_meta[~student_meta.S_LEP.isin(['998','999',999,998,' '])]
    
    teacher_meta.loc[teacher_meta['MALE'] == ' ', 'MALE'] = 0
    teacher_meta.loc[teacher_meta['BLACK'] == ' ', 'BLACK'] = 0
    teacher_meta.loc[teacher_meta['WHITE'] == ' ', 'WHITE'] = 0
    teacher_meta.loc[teacher_meta['HISP'] == ' ', 'HISP'] = 0
    teacher_meta.loc[teacher_meta['ASIAN'] == ' ', 'ASIAN'] = 0
    teacher_meta.loc[teacher_meta['EXPERIENCE'] == ' ', 'EXPERIENCE'] = 0
    teacher_meta.loc[teacher_meta['RACEOTHER'] == ' ', 'RACEOTHER'] = 0
    teacher_meta['EXPERIENCE'] = teacher_meta['EXPERIENCE'].astype(float)
    
    teacher_meta = teacher_meta[~teacher_meta.MALE.isin(['998','999',999,998,' '])]
    teacher_meta = teacher_meta[~teacher_meta.BLACK.isin(['998','999',999,998,' '])]
    teacher_meta = teacher_meta[~teacher_meta.WHITE.isin(['998','999',999,998,' '])]
    teacher_meta = teacher_meta[~teacher_meta.HISP.isin(['998','999',999,998,' '])]
    teacher_meta = teacher_meta[~teacher_meta.ASIAN.isin(['998','999',999,998,' '])]
    teacher_meta = teacher_meta[~teacher_meta.RACEOTHER.isin(['998','999',999,998,' '])]

    print("student_meta", len(student_meta), "teacher_meta", len(teacher_meta))
    # 处理MQI数据
    mqi_data = mqi_data[~mqi_data.MQI5.isin(['998','999',999,998,' '])]

    # 处理 classroom_obs
    classroom_obs = classroom_obs[~classroom_obs.CLPC.isin(['998','999',999,998,' '])]
    classroom_obs = classroom_obs[~classroom_obs.CLTS.isin(['998','999',999,998,' '])]
    classroom_obs = classroom_obs[~classroom_obs.CLRSP.isin(['998','999',999,998,' '])]
    classroom_obs = classroom_obs[~classroom_obs.CLINSTD.isin(['998','999',999,998,' '])]
    classroom_obs['CLPC'] = classroom_obs['CLPC'].astype(int)
    classroom_obs['CLTS'] = classroom_obs['CLTS'].astype(int)
    classroom_obs['CLRSP'] = classroom_obs['CLRSP'].astype(int)
    classroom_obs['CLINSTD'] = classroom_obs['CLINSTD'].astype(int)

    # Replace spaces with NaN instead of 0 to avoid bias
    teacher_va['STATEVA_M11'] = teacher_va['STATEVA_M11'].replace(' ', np.nan)
    teacher_va['STATEVA_M12'] = teacher_va['STATEVA_M12'].replace(' ', np.nan)
    teacher_va['STATEVA_M13'] = teacher_va['STATEVA_M13'].replace(' ', np.nan)
    teacher_va['STATEVA_M11'] = teacher_va['STATEVA_M11'].astype(float)
    teacher_va['STATEVA_M12'] = teacher_va['STATEVA_M12'].astype(float)
    teacher_va['STATEVA_M13'] = teacher_va['STATEVA_M13'].astype(float)


    return student_meta, teacher_meta, mqi_data, classroom_obs, teacher_va

student_meta, teacher_meta, mqi_data, classroom_obs, teacher_va = process_data(student_meta, teacher_meta, mqi_data, classroom_obs, teacher_va)

student_meta 12661 teacher_meta 309


In [3]:
# create observation dict (total_turns, student_turns, total_words, student_words)
from collections import defaultdict
from tqdm import tqdm
observation_dict =  []
for obsid in tqdm(utterances['OBSID'].unique()):
    utterances_df = utterances[utterances['OBSID'] == obsid]
    total_turns = len(utterances_df)
    student_turns = len(utterances_df[utterances_df['speaker'] == 'student'])
    total_words = utterances_df['num_words'].sum()
    student_words = utterances_df[utterances_df['speaker'] == 'student']['num_words'].sum()
    observation_dict.append([obsid, total_turns, student_turns, total_words, student_words])

observation_dict = pd.DataFrame(observation_dict)
observation_dict.columns = ['OBSID', 'total_turns', 'student_turns', 'total_words', 'student_words']
print(len(observation_dict))
observation_dict.head(1)

# we should map the teacher valut data to the utterances of the same year
utterances_year_dict =  utterances.set_index('OBSID')['year'].to_dict()
teacher_val_dict = dict()
for i, row in teacher_va.groupby('NCTETID'):
    # Use nanmean to handle NaN values properly
    STATEVA_M11_mean = np.nanmean(row['STATEVA_M11']) if not row['STATEVA_M11'].isna().all() else np.nan
    STATEVA_M12_mean = np.nanmean(row['STATEVA_M12']) if not row['STATEVA_M12'].isna().all() else np.nan
    STATEVA_M13_mean = np.nanmean(row['STATEVA_M13']) if not row['STATEVA_M13'].isna().all() else np.nan
    teacher_val_dict[i] = [STATEVA_M11_mean, STATEVA_M12_mean, STATEVA_M13_mean]
teacher_val_dict[11001]

100%|██████████| 1660/1660 [00:00<00:00, 1728.18it/s]


1660


[-0.10897859931, 0.001507199951, -0.337306290865]

In [4]:
def standardize_final_dataset(df, outcome_vars):
    for var in outcome_vars:
        if var in df.columns:
            valid_data = df[var].dropna()
            mean_val = valid_data.mean()
            std_val = valid_data.std()
            df[f'z_{var}'] = (df[var] - mean_val) / std_val
    return df


def extract_coef_se(result, var_name):
    try:
        coef = float(clustered_results.summary().tables[1].data[2][1])
        se = float(clustered_results.summary().tables[1].data[2][2])
        # coef = result.params[var_name]
        # se = result.bse[var_name]
        return coef, se
    except:
        return np.nan, np.nan

## Linear regression

create data and do linear regression

In [5]:
final_result =  defaultdict(dict)
df = []
for teacher_id in mqi_data['NCTETID'].unique():
    if teacher_id not in teacher_meta['NCTETID'].unique():
        continue
    if teacher_id not in student_meta['NCTETID'].unique():
        continue
    student_metadata_df = student_meta[student_meta['NCTETID'] == teacher_id]
    teacher_metadata_df = teacher_meta[teacher_meta['NCTETID'] == teacher_id]
    s_male = student_metadata_df['S_MALE'].astype(int).mean()
    s_afam = student_metadata_df['S_AFAM'].astype(int).mean()
    s_white = student_metadata_df['S_WHITE'].astype(int).mean()
    s_hisp = student_metadata_df['S_HISP'].astype(int).mean()
    s_asian = student_metadata_df['S_ASIAN'].astype(int).mean()
    s_race_other = student_metadata_df['S_RACE_OTHER'].astype(int).mean()
    s_frpl = student_metadata_df['S_FRPL'].astype(int).mean()
    s_sped = student_metadata_df['S_SPED'].astype(int).mean()
    s_lep = student_metadata_df['S_LEP'].astype(int).mean()
    t_male = teacher_metadata_df['MALE'].astype(int).mean()
    t_black = teacher_metadata_df['BLACK'].astype(int).mean()
    t_white = teacher_metadata_df['WHITE'].astype(int).mean()
    t_hisp = teacher_metadata_df['HISP'].astype(int).mean()
    t_asian = teacher_metadata_df['ASIAN'].astype(int).mean()
    t_race_other = teacher_metadata_df['RACEOTHER'].astype(int).mean()
    t_experience = teacher_metadata_df['EXPERIENCE'].mean()

    mqi = mqi_data[mqi_data['NCTETID'] == teacher_id]
    for obsid in mqi['OBSID'].unique():
        mqi_score = mqi[mqi['OBSID'] == obsid]['MQI5'].mean()
        # print(year, obsid)
        try:
            year = utterances_year_dict[obsid]
            teacher_va_score = teacher_val_dict[teacher_id][year-1]
            # Handle NaN values
            if pd.isna(teacher_va_score):
                teacher_va_score = None
        except:
            teacher_va_score = None
        student_reasoning = reasoning[reasoning['OBSID'] == obsid]
        paired_data = paired[paired['OBSID'] == obsid]
        student_on_task = paired_data['student_on_task'].mean()
        teacher_on_task = paired_data['teacher_on_task'].mean()
        high_uptake = paired_data['high_uptake'].mean()
        focusing_question = paired_data['focusing_question'].mean()
        observation_dict_df = observation_dict[observation_dict['OBSID'] == obsid]
        if len(observation_dict_df) > 0:
            student_turn_precent = observation_dict_df['student_turns'].values[0] / observation_dict_df['total_turns'].values[0]
            student_word_precent = observation_dict_df['student_words'].values[0] / observation_dict_df['total_words'].values[0]
            observation = len(observation_dict_df)
        if len(student_reasoning) > 0 or len(paired_data) > 0 or len(observation_dict) > 0:
            df.append({'NCTETID': teacher_id,
             'MQI5': mqi_score,
             'teacher_va': teacher_va_score,
              'student_reasoning': student_reasoning.student_reasoning.mean() if len(student_reasoning) > 0 else None,
              'student_on_task': student_on_task if len(paired_data) > 0 else None,
              'teacher_on_task': teacher_on_task if len(paired_data) > 0 else None,
              'high_uptake': high_uptake if len(paired_data) > 0 else None,
              'focusing_question': focusing_question if len(paired_data) > 0 else None,
              'student_turn_precent': student_turn_precent if len(observation_dict) > 0 else None,
              'student_word_precent': student_word_precent if len(observation_dict) > 0 else None,
              'observation': observation if len(observation_dict) > 0 else None,
              's_male': s_male,
              's_afam': s_afam,
              's_white': s_white,
              's_hisp': s_hisp,
              's_asian': s_asian,
              's_race_other': s_race_other,
              's_frpl': s_frpl,
              's_sped': s_sped,
              's_lep': s_lep,
              't_male': t_male,
              't_black': t_black,
              't_white': t_white,
              't_hisp': t_hisp,
              't_asian': t_asian,
              't_race_other': t_race_other,
              't_experience': t_experience,
              })
df = pd.DataFrame(df)
print("concat df length : ", len(df))
df = df.drop_duplicates()
print("drop duplicates df length : ", len(df))
print("standardize  MQI5 and teacher_va")
df = standardize_final_dataset(df, ['MQI5', 'teacher_va'])

import statsmodels.formula.api as sm
# teacher demographic : gender(binary), race(multi-class), experience(continuous)
# student demographic : gender(binary), race(multi-class), free lunch(binary), special education(binary), english learner(binary)

# MQI regression
for xlist in ['student_reasoning', 'student_on_task', 'teacher_on_task', 'high_uptake', 'focusing_question', 'student_turn_precent', 'student_word_precent']:
    df_mqi = df[df[xlist].notna() & df['z_MQI5'].notna()]
    print(f"MQI {xlist} {len(df_mqi)} ")
    result = sm.ols(
        formula=
    """z_MQI5 ~ {} + t_male + t_experience + t_race_other + s_male + s_white + s_asian + s_hisp + s_frpl + s_sped + s_lep """.format(xlist), data=df_mqi).fit()
    clustered_results = result.get_robustcov_results(cov_type='cluster', groups=df_mqi['NCTETID'])
    coef, std_err = extract_coef_se(clustered_results, xlist)
    final_result[xlist]['MQI'] = [coef, std_err]  

# teacher value added regression
df_va = df[df['teacher_va'].notna()].copy()
print("doing teacher_va df length : ", len(df_va))
# z-scale the teacher_va
for xlist in ['student_reasoning', 'student_on_task', 'teacher_on_task', 'high_uptake', 'focusing_question', 'student_turn_precent', 'student_word_precent']:
    df_val = df_va[df_va[xlist].notna() & df_va['z_teacher_va'].notna()]
    print(f"teacher_va {xlist} {len(df_val)}")
    result = sm.ols(
        formula=
    """z_teacher_va ~ {} + t_male + t_experience + t_race_other + s_male + s_white + s_asian + s_hisp + s_frpl + s_sped + s_lep """.format(xlist), data=df_val).fit()
    clustered_results = result.get_robustcov_results(cov_type='cluster', groups=df_val['NCTETID'])
    coef, std_err = extract_coef_se(clustered_results, xlist)
    final_result[xlist]['teacher_va'] = [coef, std_err]  

concat df length :  1631
drop duplicates df length :  1630
standardize  MQI5 and teacher_va
MQI student_reasoning 717 
MQI student_on_task 744 
MQI teacher_on_task 744 
MQI high_uptake 744 
MQI focusing_question 744 
MQI student_turn_precent 1630 
MQI student_word_precent 1630 
doing teacher_va df length :  1523
teacher_va student_reasoning 684
teacher_va student_on_task 707
teacher_va teacher_on_task 707
teacher_va high_uptake 707
teacher_va focusing_question 707
teacher_va student_turn_precent 1523
teacher_va student_word_precent 1523


In [6]:
df = []
for teacher_id in classroom_obs['NCTETID'].unique():
    if teacher_id not in teacher_meta['NCTETID'].unique():
        continue
    if teacher_id not in student_meta['NCTETID'].unique():
        continue
    student_metadata_df = student_meta[student_meta['NCTETID'] == teacher_id]
    teacher_metadata_df = teacher_meta[teacher_meta['NCTETID'] == teacher_id]
    s_male = student_metadata_df['S_MALE'].astype(int).mean()
    s_afam = student_metadata_df['S_AFAM'].astype(int).mean()
    s_white = student_metadata_df['S_WHITE'].astype(int).mean()
    s_hisp = student_metadata_df['S_HISP'].astype(int).mean()
    s_asian = student_metadata_df['S_ASIAN'].astype(int).mean()
    s_race_other = student_metadata_df['S_RACE_OTHER'].astype(int).mean()
    s_frpl = student_metadata_df['S_FRPL'].astype(int).mean()
    s_sped = student_metadata_df['S_SPED'].astype(int).mean()
    s_lep = student_metadata_df['S_LEP'].astype(int).mean()
    t_male = teacher_metadata_df['MALE'].astype(int).mean()
    t_black = teacher_metadata_df['BLACK'].astype(int).mean()
    t_white = teacher_metadata_df['WHITE'].astype(int).mean()
    t_hisp = teacher_metadata_df['HISP'].astype(int).mean()
    t_asian = teacher_metadata_df['ASIAN'].astype(int).mean()
    t_race_other = teacher_metadata_df['RACEOTHER'].astype(int).mean()
    t_experience = teacher_metadata_df['EXPERIENCE'].mean()

    classroom_obs_df = classroom_obs[classroom_obs['NCTETID'] == teacher_id]
    for obsid in classroom_obs_df['OBSID'].unique():
        
        clpc = classroom_obs_df[classroom_obs_df['OBSID'] == obsid]['CLPC'].mean()
        clts = classroom_obs_df[classroom_obs_df['OBSID'] == obsid]['CLTS'].mean()
        clrsp = classroom_obs_df[classroom_obs_df['OBSID'] == obsid]['CLRSP'].mean()
        clinstd = classroom_obs_df[classroom_obs_df['OBSID'] == obsid]['CLINSTD'].mean()
        student_reasoning = reasoning[reasoning['OBSID'] == obsid]
        paired_data = paired[paired['OBSID'] == obsid]
        student_on_task = paired_data['student_on_task'].mean()
        teacher_on_task = paired_data['teacher_on_task'].mean()
        high_uptake = paired_data['high_uptake'].mean()
        focusing_question = paired_data['focusing_question'].mean()
        observation_dict_df = observation_dict[observation_dict['OBSID'] == obsid]
        if len(observation_dict_df) > 0:
            student_turn_precent = observation_dict_df['student_turns'].values[0] / observation_dict_df['total_turns'].values[0]
            student_word_precent = observation_dict_df['student_words'].values[0] / observation_dict_df['total_words'].values[0]
            observation = len(observation_dict_df)
        if len(student_reasoning) > 0 or len(paired_data) > 0 or len(observation_dict) > 0:
            df.append({'NCTETID': teacher_id,
            #  'MQI5': mqi_score,
            'clpc': clpc,
            'clts': clts,
            'clrsp': clrsp,
            'clinstd': clinstd,
              'student_reasoning': student_reasoning.student_reasoning.mean() if len(student_reasoning) > 0 else None,
              'student_on_task': student_on_task if len(paired_data) > 0 else None,
              'teacher_on_task': teacher_on_task if len(paired_data) > 0 else None,
              'high_uptake': high_uptake if len(paired_data) > 0 else None,
              'focusing_question': focusing_question if len(paired_data) > 0 else None,
              'student_turn_precent': student_turn_precent if len(observation_dict) > 0 else None,
              'student_word_precent': student_word_precent if len(observation_dict) > 0 else None,
              'observation': observation if len(observation_dict) > 0 else None,
              's_male': s_male,
              's_afam': s_afam,
              's_white': s_white,
              's_hisp': s_hisp,
              's_asian': s_asian,
              's_race_other': s_race_other,
              's_frpl': s_frpl,
              's_sped': s_sped,
              's_lep': s_lep,
              't_male': t_male,
              't_black': t_black,
              't_white': t_white,
              't_hisp': t_hisp,
              't_asian': t_asian,
              't_race_other': t_race_other,
              't_experience': t_experience,
              })
df = pd.DataFrame(df)
df = df.drop_duplicates()
print("drop duplicates df length : ", len(df))
print("standardize clpc, clts, clrsp, clinstd")
df = standardize_final_dataset(df, ['clpc', 'clts', 'clrsp', 'clinstd'])
import statsmodels.formula.api as sm


# teacher demographic : gender(binary), race(multi-class), experience(continuous)
# student demographic : gender(binary), race(multi-class), free lunch(binary), special education(binary), english learner(binary)

for ylist in ['clpc', 'clts', 'clrsp', 'clinstd']:
    for xlist in ['student_reasoning', 'student_on_task', 'teacher_on_task', 'high_uptake', 'focusing_question', 'student_turn_precent', 'student_word_precent']:
        df_mqi = df[df[xlist].notna() & df[f'z_{ylist}'].notna()]
        print(f"{ylist} {xlist} {len(df_mqi)}")
        result = sm.ols(
            formula=
        """ z_{} ~ {} + t_male + t_experience + t_race_other + s_male + s_white + s_asian + s_hisp + s_frpl + s_sped + s_lep """.format(ylist, xlist), data=df_mqi).fit()
        clustered_results = result.get_robustcov_results(cov_type='cluster', groups=df_mqi['NCTETID'])
        coef, std_err = extract_coef_se(clustered_results, xlist)
        final_result[xlist][ylist] = [coef, std_err]
# final_result


drop duplicates df length :  1647
standardize clpc, clts, clrsp, clinstd
clpc student_reasoning 717
clpc student_on_task 744
clpc teacher_on_task 744
clpc high_uptake 744
clpc focusing_question 744
clpc student_turn_precent 1647
clpc student_word_precent 1647
clts student_reasoning 717
clts student_on_task 744
clts teacher_on_task 744
clts high_uptake 744
clts focusing_question 744
clts student_turn_precent 1647
clts student_word_precent 1647
clrsp student_reasoning 717
clrsp student_on_task 744
clrsp teacher_on_task 744
clrsp high_uptake 744
clrsp focusing_question 744
clrsp student_turn_precent 1647
clrsp student_word_precent 1647
clinstd student_reasoning 717
clinstd student_on_task 744
clinstd teacher_on_task 744
clinstd high_uptake 744
clinstd focusing_question 744
clinstd student_turn_precent 1647
clinstd student_word_precent 1647


In [7]:
final_result_list = []
for xlist in final_result.keys():
    one_list = [xlist]
    keys = ['teacher_va','MQI', 'clinstd','clts',  'clrsp',  'clpc']
    for ylist in keys:
        one_list.append(final_result[xlist][ylist][0])
        one_list.append(final_result[xlist][ylist][1])
    final_result_list.append(one_list)
final_result_list = pd.DataFrame(final_result_list,columns=['x','teacher_va_coef','teacher_va_std','MQI_coef','MQI_std','clinstd_coef','clinstd_std','clts_coef','clts_std','clrsp_coef','clrsp_std','clpc_coef','clpc_std'])
final_result_list = final_result_list.round(3)
x_map = {
    'student_reasoning': 'Student Reasoning(1_Our)',
    'student_on_task': 'Student on Task(1_Our)',
    'teacher_on_task': 'Teacher on Task(1_Our)',
    'high_uptake': 'Teacher Uptake(1_Our)',
    'focusing_question': 'Focusing Question(1_Our)',
    'student_turn_precent': 'Student Turn(1_Our)',
    'student_word_precent': 'Student Word(1_Our)',
    'observation': 'Observation'
}
final_result_list['x'] = final_result_list['x'].map(x_map)

In [8]:
table5_result = pd.read_csv('paper/table5.csv',sep='\t')
# concat the table5_result and final_result_list, in axis 0
merged_result = pd.concat([table5_result, final_result_list], axis=0)
merged_result = merged_result.sort_values(by='x').reset_index(drop=True).round(3)
def convert_dataframe_to_markdown(df, filename):
    # left align the first column
    # precision 3, zero decimal
    markdown_table = df.to_markdown(index=False, stralign='center')

    with open(filename, 'w') as f:
        f.write(markdown_table)

convert_dataframe_to_markdown(merged_result, 'merged_result.md')

In [9]:
merged_result

,x,teacher_va_coef,teacher_va_std,MQI_coef,MQI_std,clinstd_coef,clinstd_std,clts_coef,clts_std,clrsp_coef,clrsp_std,clpc_coef,clpc_std
0,Focusing Question(0_Paper),0.121*,-0.050,0.117**,-0.032,0.083**,-0.026,0.089**,-0.019,0.058**,-0.017,0.079**,-0.017
1,Focusing Question(1_Our),0.263,0.121,0.132,0.150,0.163,0.130,-0.093,0.138,0.115,0.132,0.048,0.134
2,Student Reasoning(0_Paper),0.191*,-0.091,0.313**,-0.066,0.246**,-0.050,0.144**,-0.031,0.173**,-0.035,0.120**,-0.035
3,Student Reasoning(1_Our),-0.067,0.127,0.436,0.133,0.376,0.105,-0.016,0.105,0.163,0.123,-0.127,0.116
4,Student Turn(0_Paper),1.044,-1.357,-0.047,-0.528,0.718,-0.669,0.214,-0.574,0.125,-0.485,-0.172,-0.560
5,Student Turn(1_Our),-0.08,0.416,1.024,0.365,-0.008,0.386,-0.662,0.376,-0.005,0.359,-1.281,0.364
6,Student Word(0_Paper),0.359,-0.792,0.721+,-0.413,1.132*,-0.541,0.001,-0.325,0.469,-0.395,0.322,-0.387
7,Student Word(1_Our),0.478,0.516,0.7,0.449,1.161,0.509,-0.075,0.337,0.501,0.387,0.023,0.420
8,Student on Task(0_Paper),0.038+,-0.020,0.022*,-0.010,0.032**,-0.011,0.033**,-0.008,0.024**,-0.007,0.036**,-0.007
9,Student on Task(1_Our),0.342,0.120,0.332,0.126,0.256,0.129,0.254,0.120,-0.107,0.126,0.218,0.115
